In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

# Replace this with your Spark Master URL
spark_master_url = "spark://192.168.18.110:7077"

# Initialize Spark session
spark = SparkSession.builder \
    .master(spark_master_url) \
    .appName("Read CSV from MinIO bucket") \
    .config("spark.hadoop.fs.s3a.access.key", "GD5Lg2x7tLaGxShjozFw") \
    .config("spark.hadoop.fs.s3a.secret.key", "H7J3mn7pI6oK35Xpuyrv6Lk4AsgymaSiQ2zwdAlu") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://192.168.18.115:9000") \
    .getOrCreate()    

# Access the SparkContext from the SparkSession
sc = spark.sparkContext

# Check if the application is connected to the Spark master
def check_spark_connection():
    try:
        # Access the master URL and application ID
        master_url = sc.master
        app_id = sc.applicationId
        
        print(f"Connected to Spark Master at: {master_url}")
        print(f"Application ID: {app_id}")
        return True
    except Exception as e:
        print("Failed to connect to the Spark cluster.")
        print(f"Error: {e}")
        return False

# Run the connection check
if check_spark_connection():
    print("Spark connection successful.")
else:
    print("Failed to connect to Spark. Exiting...")
    spark.stop()  # Stop the Spark session if connection fails
    exit(1)

Connected to Spark Master at: spark://192.168.18.110:7077
Application ID: app-20241113141202-0003
Spark connection successful.


In [2]:
# Specify the S3 file path
s3_csv_path = "s3a://spark/allsales.csv"

# Read the CSV file
df = spark.read.csv(s3_csv_path, header=True, inferSchema=True)

# Show the DataFrame
df.show()

+-------+-------+----------+--------+------+----+-------+
|product|country|      date|quantity|amount|card|Cust_ID|
+-------+-------+----------+--------+------+----+-------+
| prod_1|  China|12.12.2009|       1|    35|   Y| Cust_2|
| prod_2|Germany|01.02.2011|       1|    40|   Y| Cust_1|
| prod_3|    USA|17.03.2010|       1|    80|   Y| Cust_3|
| prod_1|  China|28.06.2010|      10|   350|   Y| Cust_5|
| prod_2|Germany|31.03.2010|       5|   200|   Y| Cust_4|
| prod_3|    USA|20.08.2009|      20|  1600|NULL| Cust_3|
| prod_1|    USA|11.10.2010|       2|    70|   Y| Cust_6|
| prod_2|Germany|22.11.2009|      15|   600|   N| Cust_1|
| prod_3|Germany|13.01.2010|       1|    80|NULL| Cust_4|
| prod_1|    USA|04.07.2009|       2|    70|   Y| Cust_3|
| prod_2|    USA|20.01.2010|       2|    80|NULL| Cust_6|
| prod_3|Germany|14.09.2010|       2|   160|NULL| Cust_1|
| prod_1| Brazil|17.07.2010|       5|   175|NULL| Cust_7|
| prod_2|    USA|07.07.2010|      12|   480|NULL| Cust_3|
| prod_4|unkno

In [3]:
df.createOrReplaceTempView("sales_data")
result = spark.sql("SELECT * FROM sales_data WHERE amount > 100")
result.show()

+-------+-------+----------+--------+------+----+-------+
|product|country|      date|quantity|amount|card|Cust_ID|
+-------+-------+----------+--------+------+----+-------+
| prod_1|  China|28.06.2010|      10|   350|   Y| Cust_5|
| prod_2|Germany|31.03.2010|       5|   200|   Y| Cust_4|
| prod_3|    USA|20.08.2009|      20|  1600|NULL| Cust_3|
| prod_2|Germany|22.11.2009|      15|   600|   N| Cust_1|
| prod_3|Germany|14.09.2010|       2|   160|NULL| Cust_1|
| prod_1| Brazil|17.07.2010|       5|   175|NULL| Cust_7|
| prod_2|    USA|07.07.2010|      12|   480|NULL| Cust_3|
| prod_3|  China|02.01.2011|       8|   640|NULL| Cust_2|
| prod_1|Germany|20.03.2011|      11|   385|   N| Cust_4|
| prod_2|    USA|22.02.2010|       6|   240|NULL| Cust_6|
| prod_3|  China|10.05.2009|       2|   160|NULL| Cust_2|
| prod_1|Germany|06.03.2011|      10|   350|NULL| Cust_4|
| prod_2|Germany|22.06.2010|       6|   240|NULL| Cust_1|
| prod_3| Brazil|08.06.2009|      15|  1200|NULL| Cust_7|
| prod_1|  Chi